### Creating a logistic regression to predict absenteeism 

In [1]:
import numpy as np
import pandas as pd 

In [2]:
data = pd.read_csv('Absenteeism_preprocessed.csv')
data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [3]:
data['Absenteeism Time in Hours'].median()

3.0

In [4]:
targets = np.where(data['Absenteeism Time in Hours']>data['Absenteeism Time in Hours'].median(), 1, 0)
targets.shape

(700,)

In [5]:
data['Excessive Absenteeism'] = targets
data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [6]:
targets.sum() / targets.shape[0]

0.45571428571428574

In [7]:
data = data.drop(['Absenteeism Time in Hours',
       'Day of the week', 'Distance to Work','Daily Work Load Average'], axis = 1)
data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


### Inputs for the regression

In [8]:
unscaled_inputs = data.iloc[:,:-1]

In [9]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# scaler.fit(unscaled_inputs)

#### Standardizing the inputs and not dummy variables

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self,columns):
        self.scaler = StandardScaler()
        self.columns = columns
        self.mean_ = None
        self.var_ = None
 
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.array(np.mean(X[self.columns]))
        self.var_ = np.array(np.var(X[self.columns]))
        return self
 
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), 
        columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [11]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [12]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [13]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [14]:
scaler = CustomScaler(columns_to_scale)

In [15]:
scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'])

In [16]:
scaled_inputs = scaler.transform(unscaled_inputs) 
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


### Splitting the data into training and test dataset

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_test_split(scaled_inputs, targets)

[     Reason_1  Reason_2  Reason_3  Reason_4  Month Value  \
 252         0         0         1         0     0.468236   
 370         0         0         0         1    -1.244823   
 580         0         0         0         1    -1.530333   
 337         0         0         0         0     1.324766   
 554         0         0         0         1     1.324766   
 ..        ...       ...       ...       ...          ...   
 325         0         0         0         0     1.324766   
 163         1         0         0         0    -0.959313   
 502         0         0         0         1     0.753746   
 93          0         0         0         1     1.324766   
 355         0         0         0         1     1.610276   
 
      Transportation Expense       Age  Body Mass Index  Education  Children  \
 252                1.005844 -0.536062         0.767431          0  0.880469   
 370               -0.654143  0.248310         1.002633          0 -0.919030   
 580               -0.6541

In [19]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((560, 11), (140, 11), (560,), (140,))

### Logistic regression with sklearn

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#### Training the model

In [21]:
reg = LogisticRegression()
reg.fit(x_train,y_train)

LogisticRegression()

In [22]:
reg.score(x_train, y_train)   

0.7732142857142857

### Finding the intercept and coefficients

In [23]:
intercept = reg.intercept_ 
intercept

array([-1.6474549])

In [24]:
coefficients = reg.coef_
coefficients

array([[ 2.80019733,  0.95188356,  3.11555338,  0.83900082,  0.1589299 ,
         0.60528415, -0.16989096,  0.27981088, -0.21053312,  0.34826214,
        -0.27739602]])

In [25]:
feature_name = unscaled_inputs.columns.values

In [26]:
summary_table = pd.DataFrame ( columns = ['Feature name'], data = feature_name)

summary_table['Coefficient'] = np.transpose(coefficients)

summary_table

,Feature name,Coefficient
0,Reason_1,2.800197
1,Reason_2,0.951884
2,Reason_3,3.115553
3,Reason_4,0.839001
4,Month Value,0.158930
5,Transportation Expense,0.605284
6,Age,-0.169891
7,Body Mass Index,0.279811
8,Education,-0.210533
9,Children,0.348262


In [27]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', intercept[0]]

In [28]:
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-1.647455
1,Reason_1,2.800197
2,Reason_2,0.951884
3,Reason_3,3.115553
4,Reason_4,0.839001
5,Month Value,0.158930
6,Transportation Expense,0.605284
7,Age,-0.169891
8,Body Mass Index,0.279811
9,Education,-0.210533


In [29]:
summary_table['Odds Ratio'] = np.exp(summary_table.Coefficient)
summary_table

,Feature name,Coefficient,Odds Ratio
0,Intercept,-1.647455,0.192539
1,Reason_1,2.800197,16.447892
2,Reason_2,0.951884,2.590585
3,Reason_3,3.115553,22.545903
4,Reason_4,0.839001,2.314054
5,Month Value,0.158930,1.172256
6,Transportation Expense,0.605284,1.831773
7,Age,-0.169891,0.843757
8,Body Mass Index,0.279811,1.322880
9,Education,-0.210533,0.810152


In [30]:
summary_table.sort_values('Odds Ratio', ascending = False)

,Feature name,Coefficient,Odds Ratio
3,Reason_3,3.115553,22.545903
1,Reason_1,2.800197,16.447892
2,Reason_2,0.951884,2.590585
4,Reason_4,0.839001,2.314054
6,Transportation Expense,0.605284,1.831773
10,Children,0.348262,1.416604
8,Body Mass Index,0.279811,1.322880
5,Month Value,0.158930,1.172256
7,Age,-0.169891,0.843757
9,Education,-0.210533,0.810152


### Testing the model

In [31]:
reg.score(x_test, y_test)

0.75

In [32]:
predicted_probability = reg.predict_proba(x_test)
predicted_probability.shape

(140, 2)

In [33]:
predicted_probability[:,1]

array([0.28659587, 0.41275772, 0.55979179, 0.21840536, 0.91589146,
       0.66512397, 0.70015424, 0.86896029, 0.21374596, 0.25096368,
       0.50602402, 0.77515087, 0.92870849, 0.26821867, 0.69065865,
       0.4528329 , 0.44947725, 0.4607293 , 0.59798883, 0.94638425,
       0.2996991 , 0.21840536, 0.57962872, 0.57962872, 0.75216435,
       0.25433741, 0.48982726, 0.14309805, 0.79650267, 0.21840536,
       0.36956558, 0.67906035, 0.68502567, 0.52868083, 0.21840536,
       0.53506551, 0.22147081, 0.73692105, 0.40498044, 0.60505988,
       0.21075848, 0.45224466, 0.23751292, 0.39833498, 0.82755447,
       0.56797575, 0.69113325, 0.28659587, 0.21935267, 0.2033097 ,
       0.57628256, 0.3294664 , 0.66512397, 0.26949499, 0.83321968,
       0.43491525, 0.88374612, 0.23127072, 0.33415858, 0.34432939,
       0.69909345, 0.65494263, 0.29244941, 0.79200758, 0.20750276,
       0.26840558, 0.08708566, 0.22147081, 0.73245417, 0.30530219,
       0.22147081, 0.29014408, 0.90438191, 0.46061297, 0.60174

## Saving the model

In [34]:
import pickle

In [35]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [36]:
with open('scaler', 'wb') as file:
    pickle.dump(scaler, file)